## Spark

In the same way that for many programs hellow word is the first program in spark is computing pi


In [1]:
import random
from pyspark import SparkContext
sc = SparkContext()
NUM_SAMPLES = 100000000
def inside(p):
 x, y = random.random(), random.random()
 return x*x + y*y < 1
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/21 16:49:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Pi is roughly 3.1415884


## SQL and DataFrames

There are two approaches to Spark, the DataFrame approach and the RDD approach. We are going to learn the SQL approach since it is works in the way spark intends.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Session

Spark needs to use a session in order to process data in a parallel way.

A session can be built in many different ways, what is going to be a difference maker for most local machines is that we need to specify to spark to either get it or create it.

We will use this session to define our Spark DataFrames.

When Creating DataFrames we can let spark infer the schema.

In [5]:
spark = SparkSession.builder.appName('emotion').getOrCreate()
df = spark.read.csv('iris.csv', header = True, inferSchema = True)
df.printSchema()

AnalysisException: Path does not exist: file:/Users/felixschekerka/Desktop/Strive School/ai_feb22_felix/Chapter 04/D13. Spark/iris.csv

Or we can create our own schema

In [ ]:
table_schema = StructType([
                     StructField('sepal_length', DoubleType(), False),
                     StructField('sepal_width', DoubleType(), False),
                     StructField('petal_length', DoubleType(), False),
                     StructField('petal_width', DoubleType(), False),
                     StructField('type', StringType(), False)])

In [ ]:
df2 = spark.read.csv('iris.csv', header = True, schema = table_schema)
df2.printSchema()

## Features

Spark doesn`t need the X and Y separated in the standard format.

As you will see throught the notebook spark will directly operate on the DataFrame after specifying an input and aoutput column.

By default Spark takes a column called deatures as the input in all classifiers and the Y column is called labels

We can create the  feature column by using a vector assembler

In [ ]:
numericCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

## Encoding

We can use a string indexes in the same way as the vector assembler to ordinally encode our types

In [ ]:
label_stringIdx = StringIndexer(inputCol = 'type', outputCol = 'labelIndex')
df = label_stringIdx.fit(df).transform(df)
df.show(10)

## Train/Test split

Spark Dataframes come pre-equipped with a random split function that will give you as many portions as specified.

The proportions for each portion are passed in a list

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 0)

## Classifiers

Many Spark classifiers unfortunatelly do not handle good multylabel classification so be very carefull with which you choose.

they can all be found here: https://spark.apache.org/docs/latest/ml-classification-regression.html

## Specifying input and target

As I said the default names are features an label, but we can also specify them

In [ ]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'labelIndex')

## Fitting

It is done in the same way as SKlearn

In [ ]:
rfModel = rf.fit(train)

## Predictions
    
This part is a little different. Spark will not output a prediction vector, it will direclty add a column to the DataFrame.

To predict we call the method 'transform' from the classifier

In [ ]:
predictions = rfModel.transform(test)

## Selecting

This structures are built to be parallelized in the CPU so we cannot access them in a standard fashion.

To get a subset of columns we need to use select

In [ ]:
predictions.select('sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'labelIndex', 'rawPrediction', 'prediction', 'probability').show(25)

In [ ]:
predictions.select("labelIndex", "prediction").show(10)

## Evaluating

In a very familiar format to the rest of the operations.

We define the evaluator of choice and specify which columns it should look at, labelCol and predictionCol.

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))